# Predicción de Default de Clientes con Regresión Logística

En este cuaderno desarrolla un modelo de clasificación para predecir si un cliente
incurrirá en default el próximo mes. El dataset incluye 23 variables relacionadas
con historial crediticio, pagos y características demográficas.

In [15]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import json

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    precision_score, balanced_accuracy_score, recall_score,
    f1_score, confusion_matrix
)


# Paso 1.
Realice la limpieza de los datasets:
Renombre la columna "default payment next month" a "default".
Remueva la columna "ID".
Elimine los registros con informacion no disponible.
Para la columna EDUCATION, valores > 4 indican niveles superiores
de educación, agrupe estos valores en la categoría "others".
Renombre la columna "default payment next month" a "default"
Remueva la columna "ID".

In [16]:
#carga de datos
train = pd.read_csv(
    "../files/input/train_data.csv.zip",
    compression="zip",
)
test = pd.read_csv(
    "../files/input/test_data.csv.zip",
    compression="zip",
)
print("Datos cargados correctamente.")

Datos cargados correctamente.


In [17]:
# Renombrar variable objetivo
train = train.rename(columns={"default payment next month": "default"})
test = test.rename(columns={"default payment next month": "default"})

# Eliminar columna ID 
if "ID" in train.columns:
    train = train.drop(columns=["ID"])
if "ID" in test.columns:
    test = test.drop(columns=["ID"])

# Eliminar filas con valores faltantes
train = train.dropna()
test = test.dropna()

# EDUCATION > 4 se agrupa en categoría "otros" (4)
train["EDUCATION"] = train["EDUCATION"].clip(upper=4)
test["EDUCATION"] = test["EDUCATION"].clip(upper=4)

print("Datos preprocesados correctamente.")

Datos preprocesados correctamente.


# Paso 2.
 Divida los datasets en x_train, y_train, x_test, y_test.

In [18]:
x_train = train.drop(columns=["default"])
y_train = train["default"]

x_test = test.drop(columns=["default"])
y_test = test["default"]

print("Variables independientes y dependientes separadas.")

Variables independientes y dependientes separadas.


# Paso 3.
Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:

 - Transforma las variables categoricas usando el método one-hot-encoding.
 - Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
 - Estandariza la matriz de entrada.
 - Selecciona las K columnas mas relevantes de la matrix de entrada.
 - Ajusta una maquina de vectores de soporte (svm).


In [19]:
# Variables categóricas y numéricas
categorical = ["SEX", "EDUCATION", "MARRIAGE"]
numeric = [col for col in x_train.columns if col not in categorical]

# OneHotEncoder solo sobre categóricas, resto pasa tal cual
preprocessor = ColumnTransformer(
    transformers=[
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            categorical,
        )
    ],
    remainder="passthrough",
)

print("Preprocesador creado.")  

Preprocesador creado.


# Paso 4.
- Optimice los hiperparametros del pipeline usando validación cruzada.
- Use 10 splits para la validación cruzada. Use la función de precision
- balanceada para medir la precisión del modelo.

In [20]:
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),               # contiene OneHotEncoder
        ("scaler", StandardScaler()),                 # aparece StandardScaler
        ("pca", PCA()),                               # PCA con todas las componentes
        ("selector", SelectKBest(score_func=f_classif)),
        ("svm", SVC()),
    ]
)
print("Pipeline creado.")


Pipeline creado.


In [21]:
param_grid = {
    "selector__k": [20, 25],
    "svm__C": [1, 10, 20],
    "svm__kernel": ["rbf"],
    "svm__gamma": ["scale", "auto"],
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring="balanced_accuracy",
    cv=10,        # el enunciado pide 10 folds
    n_jobs=-1,
    verbose=1,
)

grid.fit(x_train, y_train)

print("Mejores parámetros:", grid.best_params_)
print("Mejor balanced_accuracy (cv):", grid.best_score_)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Mejores parámetros: {'selector__k': 20, 'svm__C': 10, 'svm__gamma': 'auto', 'svm__kernel': 'rbf'}
Mejor balanced_accuracy (cv): 0.6455126994487348


# Paso 5.
- Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
- Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [22]:
os.makedirs("../files/models", exist_ok=True)

with gzip.open("../files/models/model.pkl.gz", "wb") as f:
    pickle.dump(grid, f)
print("Modelo guardado en 'files/models/model.pkl.gz'.")

Modelo guardado en 'files/models/model.pkl.gz'.


# Paso 6.
- Calcule las metricas de precision, precision balanceada, recall, y f1-score para los conjuntos de entrenamiento y prueba.
- Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo.
- Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. Por ejemplo:
- - {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
- - {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}
#

In [23]:
def compute_metrics(model, X, y, name):
    y_pred = model.predict(X)
    return {
        "type": "metrics",
        "dataset": name,
        "precision": float(precision_score(y, y_pred)),
        "balanced_accuracy": float(balanced_accuracy_score(y, y_pred)),
        "recall": float(recall_score(y, y_pred)),
        "f1_score": float(f1_score(y, y_pred)),
    }


def cm_to_dict(cm, name):
    return {
        "type": "cm_matrix",
        "dataset": name,
        "true_0": {
            "predicted_0": int(cm[0, 0]),
            "predicted_1": int(cm[0, 1]),
        },
        "true_1": {
            "predicted_0": int(cm[1, 0]),
            "predicted_1": int(cm[1, 1]),
        },
    }
print("Evaluando modelo en conjunto de entrenamiento...")

Evaluando modelo en conjunto de entrenamiento...


# Paso 7.
- Calcule las matrices de confusion para los conjuntos de entrenamiento y prueba. 
- Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. de entrenamiento o prueba. Por ejemplo:
- - {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
- -{'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#

In [24]:
metrics = []

# Métricas
metrics.append(compute_metrics(grid, x_train, y_train, "train"))
metrics.append(compute_metrics(grid, x_test, y_test, "test"))

# Matrices de confusión
cm_train = confusion_matrix(y_train, grid.predict(x_train))
cm_test = confusion_matrix(y_test, grid.predict(x_test))

metrics.append(cm_to_dict(cm_train, "train"))
metrics.append(cm_to_dict(cm_test, "test"))
print("Evaluación completada.")

Evaluación completada.


In [ ]:
output_path = "../files/output/"

with open(output_path, "w", encoding="utf-8") as f:
    for entry in metrics:
        f.write(json.dumps(entry) + "\n")

print("Guardado en:", output_path)


OSError: [Errno 22] Invalid argument: '../files/output/'